In [1]:
import pandas as pd
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [2]:
portal_client = get_portal_client()

In [3]:
sites_list = portal_client.query_to_pandas(
    """WITH RECURSIVE category_path AS (
    SELECT id, name, parent_id, 0 AS depth
    FROM api_clients
    WHERE id = 118

    UNION ALL

    SELECT c.id, c.name, c.parent_id, cp.depth + 1
    FROM api_clients c
    INNER JOIN category_path cp ON cp.id = c.parent_id
    )
    SELECT category_path.id AS client_id, site.id AS site_id
    FROM category_path
    INNER JOIN topaz.api_sites AS site
    ON category_path.id = site.client_id
    WHERE toc_id IS NOT NULL
    AND template_id IS NOT NULL"""
).sort_values("site_id")

In [ ]:
missing_df = pd.read_csv("data/2025-08-04 Masterfire Missing Tasks New.csv")

In [ ]:
missing_sites = []
for _, (client_id, site_id) in sites_list.iterrows():
    if site_id in missing_df["site_id"].values:
        continue
    print(site_id)
    df = portal_client.query_to_pandas(
        """SELECT
            DISTINCT test.id AS test_id
        FROM topaz.api_service_tests AS test
        INNER JOIN topaz.api_mode_changes AS mode_change
        ON test.mode_change_id = mode_change.id
        INNER JOIN topaz.api_sites AS site
        ON test.site_id = site.id
        INNER JOIN topaz.api_toc_rows AS toc
        ON (mode_change.toc_id = toc.toc_id)
        AND (
            (test.domain = toc.domain) OR
            (toc.domain IS NULL)
        )
        AND (
            (test.panel = toc.panel) OR
            (toc.panel IS NULL)
        )
        AND (
            (test.loop = toc.loop) OR
            (toc.loop IS NULL)
        )
        AND (test.address = toc.address)
        AND (test.address_type = toc.address_type)
        AND (
            (
                (
                    (test.sub_address = '') OR
                    (test.sub_address IS NULL)
                ) AND (
                    (toc.sub_address IS NULL) OR
                    (toc.sub_address = 0)
                )
            ) OR
            (test.sub_address = toc.sub_address)
        )
        AND test.created_at BETWEEN toc.created_at AND IFNULL(toc.deleted_at, CURRENT_TIMESTAMP())
        INNER JOIN topaz.api_service_tasks AS task_template
        ON task_template.template_id = site.template_id
        AND (
            (
                (task_template.asset_type = toc.lcs_device_type)
                AND (toc.lcs_device_type!= 'Other')
            )
            OR (
                toc.lcs_device_type = 'Other'
                AND (toc.type = task_template.asset_type)
            )
            OR (task_template.asset_type = '')
            OR (
                site.protocol_id = 34
                AND asset_type = 'el'
            )
            OR (
                site.protocol_id = 35
                AND asset_type = 'ext'
                AND toc.lcs_device_type != 'Other'
                AND toc.type != 'Fire Blanket'
            )
        )
        AND test.created_at BETWEEN task_template.created_at AND IFNULL(task_template.deleted_at, CURRENT_TIMESTAMP())
        AND task_template.type IN (1,3)
        LEFT JOIN topaz.api_service_tasks_record AS task_record
        ON task_record.task_id = task_template.id
        AND task_record.test_id = test.id
        WHERE task_record.id IS NULL
        AND site.id = %(site_id)s
        LIMIT 1
        """,
        {"site_id": site_id}
    )
    if df.empty:
        new_row = pd.DataFrame({"site_id": [site_id], "missing": [0]})
    else:
        print("-------> has missing tasks")
        new_row = pd.DataFrame({"site_id": [site_id], "missing": [1]})
    missing_df = pd.concat([missing_df, new_row])
    missing_df.to_csv("data/2025-08-04 Masterfire Missing Tasks New 2024.csv", index=False)

4260
